In [ ]:
import librosa.display, os, gc
import numpy as np
import matplotlib.pyplot as plt
import scaper, os
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D

In [ ]:
def label_extraction(text_file):
    filename = str(text_file)
    text_file = open(text_file, "r")
    text_file_lines = text_file.read().split('\n')
    if text_file_lines[0] == '':
        return 0
    else:
        a = text_file_lines[0].split(',')
        if a[2] == 'chainsaw':
            return 1
        else:
            return 2

In [ ]:
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import os
import numpy as np
import random
from keras.utils import to_categorical

samples = []
labels = []
images_folder = "images/"
image_list = os.listdir(images_folder)
random.seed(10)
random.shuffle(image_list)

for image in image_list:
    samples.append(img_to_array(load_img(images_folder + image, target_size = (200, 200)))) # Change Target Size accordingly to the resolution.

    labels.append(label_extraction('txt/'+image[: image.rfind('.png')] + ".txt"))
    
labels = to_categorical(labels)
        
samples = np.array(samples)
labels = np.array(labels)

eval_x, eval_y = samples[-100:], labels[-100:] # 1000 images each for Normal and Soundscape, 100 in evaluation.
print(samples.shape, labels.shape)

x_train, x_test, y_train, y_test = train_test_split(samples[:-100], labels[:-100], test_size = 0.2, random_state=31)

In [ ]:
%%time

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.callbacks import History

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = (3, 3), input_shape = (200, 200, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.2))
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(3, activation = 'softmax'))
print(model.summary())

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
# sparse_categorical_crossentropy
# categorical_crossentropy
# binary_crossentropy
history = History()
model.fit(x_train, y_train, batch_size = 32, epochs = 50, callbacks = [history], validation_data = (x_test, y_test), verbose = 1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# plot the training loss and accuracy
def plotResults(): 
    plt.figure()
    N = 50
    
    # TODO: plot the accuracy/loss variables over training time
    plt.plot(np.arange(0, N), history.history["accuracy"], label = "train_acc")
    plt.plot(np.arange(0, N), history.history["val_accuracy"], label = "val_acc")

    # make the graph understandable: 
    plt.title("Training Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Accuracy")
    plt.legend(loc="upper right")
    plt.yscale('log')
    plt.show()
    plt.figure()
    plt.plot(np.arange(0, N), history.history["loss"], label = "train_loss")
    plt.plot(np.arange(0, N), history.history["val_loss"], label = "val_loss")
    
     # make the graph understandable: 
    plt.title("Training Loss")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.yscale('log')
    plt.legend(loc="upper right")
    plt.show()

In [ ]:
model.save_weights('custom-multiclass-model.h5')
plotResults()

In [ ]:
import pandas as pd
import numpy as np
from scipy import interp

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.preprocessing import LabelBinarizer

def class_report(y_true, y_pred, y_score=None, average='micro'):
    if y_true.shape != y_pred.shape:
        print("Error! y_true %s is not the same shape as y_pred %s" % (
              y_true.shape,
              y_pred.shape)
        )
        return

    lb = LabelBinarizer()

    if len(y_true.shape) == 1:
        lb.fit(y_true)

    #Value counts of predictions
    labels, cnt = np.unique(
        y_pred,
        return_counts=True)
    n_classes = len(labels)
    pred_cnt = pd.Series(cnt, index=labels)

    metrics_summary = precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred,
            labels=labels)

    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index,
        columns=labels)

    support = class_report_df.loc['support']
    total = support.sum() 
    class_report_df['avg / total'] = avg[:-1] + [total]

    class_report_df = class_report_df.T
    class_report_df['pred'] = pred_cnt
    class_report_df['pred'].iloc[-1] = total

    if not (y_score is None):
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for label_it, label in enumerate(labels):
            fpr[label], tpr[label], _ = roc_curve(
                (y_true == label).astype(int), 
                y_score[:, label_it])

            roc_auc[label] = auc(fpr[label], tpr[label])

        if average == 'micro':
            if n_classes <= 2:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                    lb.transform(y_true).ravel(), 
                    y_score[:, 1].ravel())
            else:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                        lb.transform(y_true).ravel(), 
                        y_score.ravel())

            roc_auc["avg / total"] = auc(
                fpr["avg / total"], 
                tpr["avg / total"])

        elif average == 'macro':
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([
                fpr[i] for i in labels]
            ))

            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in labels:
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])

            # Finally average it and compute AUC
            mean_tpr /= n_classes

            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr

            roc_auc["avg / total"] = auc(fpr["macro"], tpr["macro"])

        class_report_df['AUC'] = pd.Series(roc_auc)
        
    # print out confusion matrix
    print("Confusion Matrix: \n", confusion_matrix(y_true, y_pred))
    return class_report_df


In [ ]:
import tensorflow as tf
pred_y = model.predict_classes(eval_x, verbose = 1)
new_eval_y = tf.argmax(eval_y, axis = 1)

In [ ]:
score = model.evaluate(eval_x, eval_y, verbose = 1)
print(f"Loss: {score[0]}\nAccuracy: {score[1]}")
print(class_report(new_eval_y, pred_y, y_score=None, average='micro'))